In [12]:
import pandas as pd
import numpy as np
import os
os.chdir('/kaggle/input/godaddy-microbusiness-density-forecasting/')

In [54]:
import matplotlib.pyplot as plt
from tqdm import tqdm
from  lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.cluster import KMeans
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler


In [14]:
import warnings
warnings.filterwarnings('ignore')

In [15]:
def smape(actual, predicted):
    return 100/len(actual) * np.sum(2 * np.abs(predicted - actual) / (np.abs(actual) + np.abs(predicted)))

In [16]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_census = pd.read_csv('census_starter.csv')
df_submission = pd.read_csv('sample_submission.csv')

print(f'train: {df_train.shape} \ncensus: {df_census.shape} \ntest: {df_test.shape} \nsubmission: {df_submission.shape}')

train: (122265, 7) 
census: (3142, 26) 
test: (25080, 3) 
submission: (25080, 2)


In [60]:
df_census

,pct_bb_2017,pct_bb_2018,pct_bb_2019,pct_bb_2020,pct_bb_2021,cfips,pct_college_2017,pct_college_2018,pct_college_2019,pct_college_2020,...,pct_it_workers_2019,pct_it_workers_2020,pct_it_workers_2021,median_hh_inc_2017,median_hh_inc_2018,median_hh_inc_2019,median_hh_inc_2020,median_hh_inc_2021,id,cluster
0,76.6,78.9,80.6,82.7,85.5,1001,14.5,15.9,16.1,16.7,...,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0,0,0
1,74.5,78.1,81.8,85.1,87.9,1003,20.4,20.7,21.0,20.2,...,1.4,1.0,1.3,52562,55962.0,58320,61756.0,64346.0,1,0
2,57.2,60.4,60.5,64.6,64.6,1005,7.6,7.8,7.6,7.3,...,0.8,1.1,0.8,33368,34186.0,32525,34990.0,36422.0,2,1
3,62.0,66.1,69.2,76.1,74.6,1007,8.1,7.6,6.5,7.4,...,1.6,1.7,2.1,43404,45340.0,47542,51721.0,54277.0,3,2
4,65.8,68.5,73.0,79.6,81.0,1009,8.7,8.1,8.6,8.9,...,0.9,1.1,0.9,47412,48695.0,49358,48922.0,52830.0,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,82.2,82.4,84.0,86.7,88.4,56037,15.3,15.2,14.8,13.7,...,1.0,0.9,1.0,71083,73008.0,74843,73384.0,76668.0,3137,4
3138,83.5,85.9,87.1,89.1,90.5,56039,37.7,37.8,38.9,37.2,...,1.4,1.5,2.0,80049,83831.0,84678,87053.0,94498.0,3138,4
3139,83.8,88.2,89.5,91.4,90.6,56041,11.9,10.5,11.1,12.6,...,1.4,1.7,0.9,54672,58235.0,63403,72458.0,75106.0,3139,0
3140,76.4,78.3,78.2,82.8,85.4,56043,15.4,15.0,15.4,15.0,...,0.9,0.9,1.1,51362,53426.0,54158,57306.0,62271.0,3140,2


In [17]:
df_train['dataset'] = 'train'
df_test['dataset'] = 'test'

In [18]:
df_train['year'] = pd.to_datetime(df_train['first_day_of_month']).dt.year
df_test['year'] = pd.to_datetime(df_test['first_day_of_month']).dt.year

In [19]:
df_census = df_census[df_census['cfips'].isin(df_train['cfips'].unique())]
df_census['id'] = df_census.index

In [20]:
#Let's select the optimum number of clusters for k-means classification
intertia = []

for i in tqdm(range(1, 11)):
    kmeans = KMeans(n_clusters = i, init = 'k-means++',
                    max_iter = 400, n_init = 10, random_state = 0)
    kmeans.fit(df_census.fillna(0).drop('cfips', axis = 1))
    intertia.append(kmeans.inertia_)
#Plotting the results onto a line graph to observe 'The elbow'
plt.plot(range(1, 11), intertia)
plt.title('Elbow Method')
plt.xlabel('No of Clusters')
plt.ylabel('Intertia') #within cluster sum of squares
plt.show()

In [22]:
#Applying kmeans to the dataset / Creating the kmeans classifier
kmeans = KMeans(n_clusters = 5, init = 'k-means++', max_iter = 500, 
                n_init = 10, random_state = 0)
df_census['cluster'] = kmeans.fit_predict(df_census.fillna(0).drop('cfips', axis = 1))

In [23]:
df_census_long = pd.wide_to_long(df = df_census,
                                 stubnames = ['pct_bb','pct_college','pct_foreign_born',
                                                   'pct_it_workers','median_hh_inc'], 
                                 i = 'id',
                                 j = 'year',
                                 sep = '_')

df_census_long = df_census_long.reset_index().drop(['id'],axis = 1)
df_census_long['year'] = df_census_long['year'] + 2
df_census_long = df_census_long.sort_values(['cfips','year'])
df_census_long = df_census_long.reset_index(drop = True)

df_census_long['pct_it_workers'] = df_census_long.groupby('cfips')['pct_it_workers'].ffill()
df_census_long['median_hh_inc'] = df_census_long.groupby('cfips')['median_hh_inc'].ffill()
df_census_long.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15675 entries, 0 to 15674
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   year              15675 non-null  int64  
 1   cfips             15675 non-null  int64  
 2   cluster           15675 non-null  int32  
 3   pct_bb            15675 non-null  float64
 4   pct_college       15675 non-null  float64
 5   pct_foreign_born  15675 non-null  float64
 6   pct_it_workers    15675 non-null  float64
 7   median_hh_inc     15675 non-null  float64
dtypes: float64(5), int32(1), int64(2)
memory usage: 918.6 KB


In [24]:
def lag_feature(df):
    for lag in range(1, 13):
        df[f'lag_density_{lag}'] = df.groupby('cfips')['microbusiness_density'].shift(lag)
        df[f'lag_density_{lag}'] = df.groupby('cfips')[f'lag_density_{lag}'].bfill()
        df[f'lag_density_{lag}'] = df.groupby('cfips')[f'lag_density_{lag}'].ffill()
        
    return df

In [25]:
census_train_merged = df_train.merge(df_census_long, how = 'left', on = ['cfips','year'])

#Feature Engineering
census_train_merged['first_day_of_month'] = pd.to_datetime(census_train_merged['first_day_of_month'])
census_train_merged['month'] = pd.to_datetime(census_train_merged['first_day_of_month']).dt.month_name()
census_train_merged['quarter'] = pd.to_datetime(census_train_merged['first_day_of_month']).dt.quarter.astype(str)

census_train_merged = census_train_merged[['row_id','cfips','month','quarter','year', 'cluster',
                                           'pct_bb', 'pct_college', 'pct_foreign_born', 'pct_it_workers', 'median_hh_inc',
                                           'microbusiness_density','dataset']]

census_train_merged = pd.concat([census_train_merged.drop(['month','quarter'],axis = 1), 
                                 pd.get_dummies(census_train_merged[['month','quarter']])], axis = 1)

census_train_merged.shape

(122265, 27)

In [26]:
census_test_merged = df_test.merge(df_census_long, how = 'left', on = ['cfips','year'])

#Feature Engineering
census_test_merged['first_day_of_month'] = pd.to_datetime(census_test_merged['first_day_of_month'])
census_test_merged['month'] = pd.to_datetime(census_test_merged['first_day_of_month']).dt.month_name()
census_test_merged['quarter'] = pd.to_datetime(census_test_merged['first_day_of_month']).dt.quarter.astype(str)
census_test_merged['microbusiness_density'] = np.nan

census_test_merged = census_test_merged[['row_id','cfips','month','quarter','year', 'cluster',
                                         'pct_bb', 'pct_college', 'pct_foreign_born', 'pct_it_workers', 'median_hh_inc',
                                         'microbusiness_density', 'dataset']]

census_test_merged = pd.concat([census_test_merged.drop(['month','quarter'],axis = 1), 
                                 pd.get_dummies(census_test_merged[['month','quarter']])], axis = 1)

census_test_merged.shape

(25080, 22)

In [27]:
census_test_merged[list(set(census_train_merged.columns) - set(census_test_merged.columns))] = 0
census_test_merged = census_test_merged[census_train_merged.columns]
census_test_merged.shape

(25080, 27)

In [28]:
census_all_merged = pd.concat([census_train_merged,census_test_merged])
pd.concat([census_all_merged.head(2),census_all_merged.tail(2)])

,row_id,cfips,year,cluster,pct_bb,pct_college,pct_foreign_born,pct_it_workers,median_hh_inc,microbusiness_density,...,month_June,month_March,month_May,month_November,month_October,month_September,quarter_1,quarter_2,quarter_3,quarter_4
0,1001_2019-08-01,1001,2019,0,76.6,14.5,2.1,1.3,55317.0,3.007682,...,0,0,0,0,0,0,0,0,1,0
1,1001_2019-09-01,1001,2019,0,76.6,14.5,2.1,1.3,55317.0,2.884870,...,0,0,0,0,0,1,0,0,1,0
25078,56043_2023-06-01,56043,2023,2,85.4,17.2,1.0,1.1,62271.0,NaN,...,1,0,0,0,0,0,0,1,0,0
25079,56045_2023-06-01,56045,2023,0,81.3,13.9,1.6,0.0,65566.0,NaN,...,1,0,0,0,0,0,0,1,0,0


In [29]:
census_all_merged = lag_feature(census_all_merged)
pd.concat([census_all_merged.head(2),census_all_merged.tail(2)])

,row_id,cfips,year,cluster,pct_bb,pct_college,pct_foreign_born,pct_it_workers,median_hh_inc,microbusiness_density,...,lag_density_3,lag_density_4,lag_density_5,lag_density_6,lag_density_7,lag_density_8,lag_density_9,lag_density_10,lag_density_11,lag_density_12
0,1001_2019-08-01,1001,2019,0,76.6,14.5,2.1,1.3,55317.0,3.007682,...,3.007682,3.007682,3.007682,3.007682,3.007682,3.007682,3.007682,3.007682,3.007682,3.007682
1,1001_2019-09-01,1001,2019,0,76.6,14.5,2.1,1.3,55317.0,2.884870,...,3.007682,3.007682,3.007682,3.007682,3.007682,3.007682,3.007682,3.007682,3.007682,3.007682
25078,56043_2023-06-01,56043,2023,2,85.4,17.2,1.0,1.1,62271.0,NaN,...,3.126551,3.126551,3.126551,3.126551,3.126551,3.126551,3.209264,3.209264,3.225807,3.126551
25079,56045_2023-06-01,56045,2023,0,81.3,13.9,1.6,0.0,65566.0,NaN,...,1.785395,1.785395,1.785395,1.785395,1.785395,1.785395,1.785395,1.785395,1.803249,1.803249


In [53]:
def run_model(model, name, cluster, validation = True):
    global models
    census_train_merged = census_all_merged[census_all_merged['dataset'] == 'train'].drop(['cfips','dataset'], axis = 1)
    census_test_merged = census_all_merged[census_all_merged['dataset'] == 'test'].drop(['cfips','dataset'], axis = 1)
    
    if validation:
        X_train = census_train_merged[(census_train_merged['year'] < 2022) & (census_train_merged['cluster'] == cluster)].drop('microbusiness_density', axis = 1)
        X_test = census_train_merged[(census_train_merged['year'] == 2022) & (census_train_merged['cluster'] == cluster)].drop('microbusiness_density', axis = 1)
        y_train = census_train_merged[(census_train_merged['year'] < 2022) & (census_train_merged['cluster'] == cluster)]['microbusiness_density']
        y_test = census_train_merged[(census_train_merged['year'] == 2022) & (census_train_merged['cluster'] == cluster)]['microbusiness_density']
    else:
        X_train = census_train_merged[census_train_merged['cluster'] == cluster].drop('microbusiness_density', axis = 1)
        y_train = census_train_merged[census_train_merged['cluster'] == cluster]['microbusiness_density']
        X_test = census_test_merged[census_test_merged['cluster'] == cluster].drop('microbusiness_density', axis = 1)
    
    row_ids = X_test['row_id']
    X_train = X_train.drop('row_id', axis = 1)
    X_test = X_test.drop('row_id', axis = 1)
    
    
    sc = StandardScaler()
    sc.fit(X_train)
    X_train = sc.transform(X_train)
    X_test = sc.transform(X_test)
    
    #Model Fit
    model.fit(X_train, y_train)

    #Get predictions
    y_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)
    
    #Calculate evaluation metrics
    rmse_train = mean_squared_error(y_train, y_train_pred)**(1/2) 
    smape_train = smape(y_train, y_train_pred)
    
    if validation:
        rmse_test = mean_squared_error(y_test, y_pred)**(1/2)
        smape_test = smape(y_test, y_pred)
    else:
        rmse_test = np.nan
        smape_test = np.nan

    metrics_df = pd.DataFrame([[cluster, name, rmse_train, rmse_test, smape_train, smape_test]], columns = models.columns)
    models = pd.concat([models,metrics_df], axis = 0)
    
    return row_ids, y_pred

In [55]:
validation = True

random_state = 42
models = pd.DataFrame(columns = ['cluster', 'Model', 'Train RMSE', 'Test RMSE', 'Train SMAPE', 'Test SMAPE'])
preds = pd.DataFrame()

for cluster in tqdm(census_all_merged['cluster'].unique()):
    
    lr = LinearRegression()
    xgb = XGBRegressor(random_state = random_state, n_jobs = -1)
    lgbm = LGBMRegressor(random_state = random_state, n_jobs = -1)
    knn = KNeighborsRegressor(n_jobs = -1, n_neighbors = 5)
    rf = RandomForestRegressor(random_state = random_state, n_jobs = -1)

    ##Minimum SMAPE
    model_dict = {'XGBoost': [xgb,list(),list()],
                  'LinearRegression': [lr,list(),list()],
                  'LightGBM': [lgbm,list(),list()],
                  'KNN': [knn,list(),list()],
                  'RandomForestRegressor': [rf,list(),list()]
                 }
    
    for model_name in model_dict:
        model_dict[model_name][1], model_dict[model_name][2] = run_model(model_dict[model_name][0],model_name, cluster, validation)
    
    model_cluster = models[models['cluster'] == cluster]
    if validation:
        max_smape_model = model_cluster[model_cluster['Test SMAPE'] == min(model_cluster['Test SMAPE'])]['Model'].values[0]
    else:
        max_smape_model = model_cluster[model_cluster['Train SMAPE'] == min(model_cluster['Train SMAPE'])]['Model'].values[0]
    row_ids = model_dict[max_smape_model][1]
    best_preds = model_dict[max_smape_model][2]
    preds = preds.append([[cluster,max_smape_model,row_ids,best_preds]])

100%|██████████| 5/5 [04:26<00:00, 53.21s/it]


In [56]:
if validation:
    print(models.reset_index(drop = True).sort_values(['cluster','Test SMAPE']).groupby('cluster').head(1))
else:
    print(models.reset_index(drop = True).sort_values(['cluster','Train SMAPE']).groupby('cluster').head(1))

   cluster                  Model  Train RMSE  Test RMSE  Train SMAPE  \
1        0       LinearRegression    0.620410   1.157780     1.891627   
6        1       LinearRegression    0.202630   0.151927     3.031825   
11       2       LinearRegression    0.900718   1.964916     2.219738   
24       3  RandomForestRegressor    0.121417   0.281673     0.394568   
19       4  RandomForestRegressor    2.221518   0.313861     0.719824   

    Test SMAPE  
1     1.593379  
6     2.580983  
11    1.783070  
24    1.135724  
19    1.630229  


In [57]:
preds = preds.reset_index(drop = True)
preds.columns = ['cluster','model','row_id','predictions']
preds = preds.explode(['row_id','predictions'])
preds

,cluster,model,row_id,predictions
0,0,LinearRegression,1001_2022-01-01,3.222137
0,0,LinearRegression,1001_2022-02-01,3.332292
0,0,LinearRegression,1001_2022-03-01,3.364859
0,0,LinearRegression,1001_2022-04-01,3.386774
0,0,LinearRegression,1001_2022-05-01,3.384154
...,...,...,...,...
4,3,RandomForestRegressor,53033_2022-06-01,13.650293
4,3,RandomForestRegressor,53033_2022-07-01,13.641976
4,3,RandomForestRegressor,53033_2022-08-01,13.598855
4,3,RandomForestRegressor,53033_2022-09-01,13.67756


In [58]:
if validation:
    preds = preds.merge(df_train[['row_id','microbusiness_density']], how = 'left', on = 'row_id')
    print(f"Test SMAPE: {smape(preds['microbusiness_density'],preds['predictions'])}")
else:
    submission = preds[['row_id','predictions']]
    submission.columns = ['row_id','microbusiness_density']
    submission = submission.reset_index(drop = True)
    print(submission)
    submission.to_csv('/kaggle/working/submission_13.csv', index = False)

Test SMAPE: 1.9086600688269393
